In [1]:
import pandas as pd 
import numpy as np 
import matplotlib.pyplot as plt 
import optuna 
import lightgbm as lgb 
from sklearn.metrics import accuracy_score, roc_auc_score,precision_score, confusion_matrix,recall_score 
from sklearn.model_selection import cross_val_score, train_test_split
import os 

In [3]:
train_data=pd.read_csv('/home/iai/son/lastcheck/spambase_train_data.csv')
test_data=pd.read_csv('/home/iai/son/lastcheck/spambase_test_data.csv')


In [4]:
test_stage_features=['0.7', '278', '61', '0', '0.34', '0.30', '0.5', '0.44', '0.40', '0.778',
                        '0.24', '1.93', '0.39', '1.29', '0.43', '0.2', '0.36', '0.17', '0.20']

In [5]:
train_X=train_data[test_stage_features]
train_y=train_data['1']

new_test_data_X=test_data[test_stage_features]
new_test_data_y=test_data['1']

In [6]:
train_y.value_counts()

0    2258
1    1422
Name: 1, dtype: int64

In [7]:
ratio=train_y.value_counts()[0]/train_y.value_counts()[1]

In [8]:
def objective(trial):
    # Define hyperparameters to optimize 
    params={
        'boosting_type':'gbdt',
        'objective':'binary',
        'metric':'binary_logloss',
        'learning_rate':trial.suggest_loguniform('learning_rate',0.01,0.1),
        'max_depth':trial.suggest_int('max_depth',3,9),
        'n_estimators':trial.suggest_int("n_estimators",100,1000)
    }
    # Train and evaluate model 
    lgb_cv=lgb.LGBMClassifier(**params, random_state=42,scale_pos_weight=ratio)
    scores=cross_val_score(lgb_cv,train_X,train_y,cv=5,scoring='roc_auc')
    auc=scores.mean()
    return auc 

# Define study object and optimize 

study=optuna.create_study(direction='maximize',study_name='lgb_boost_opt',load_if_exists=True)
study.optimize(objective, n_trials=50)

# Print best hyperparameters and auc
print(f'Best hyperparameters: {study.best_params}')
print(f'Best AUC: {study.best_value:.4f}')

[I 2023-04-25 20:26:33,966] A new study created in memory with name: lgb_boost_opt
/tmp/ipykernel_1040697/2141867322.py:7: FutureWarning: suggest_loguniform has been deprecated in v3.0.0. This feature will be removed in v6.0.0. See https://github.com/optuna/optuna/releases/tag/v3.0.0. Use :func:`~optuna.trial.Trial.suggest_float` instead.
  'learning_rate':trial.suggest_loguniform('learning_rate',0.01,0.1),
[I 2023-04-25 20:26:34,648] Trial 0 finished with value: 0.9564963685748049 and parameters: {'learning_rate': 0.013254535414338622, 'max_depth': 4, 'n_estimators': 243}. Best is trial 0 with value: 0.9564963685748049.
/tmp/ipykernel_1040697/2141867322.py:7: FutureWarning: suggest_loguniform has been deprecated in v3.0.0. This feature will be removed in v6.0.0. See https://github.com/optuna/optuna/releases/tag/v3.0.0. Use :func:`~optuna.trial.Trial.suggest_float` instead.
  'learning_rate':trial.suggest_loguniform('learning_rate',0.01,0.1),
[I 2023-04-25 20:26:35,049] Trial 1 finishe

Best hyperparameters: {'learning_rate': 0.033884676814068496, 'max_depth': 6, 'n_estimators': 474}
Best AUC: 0.9680


In [9]:
lgb=lgb.LGBMClassifier(learning_rate=0.033884676814068496,max_depth=6,n_estimators=474,scale_pos_weight=ratio,random_state=42)

In [10]:
lgb.fit(train_X,train_y)

LGBMClassifier(learning_rate=0.033884676814068496, max_depth=6,
               n_estimators=474, random_state=42,
               scale_pos_weight=1.5879043600562588)

In [11]:
prob=lgb.predict_proba(new_test_data_X)[:,1]

In [12]:
from sklearn.metrics import accuracy_score, precision_score, recall_score, confusion_matrix, f1_score

def get_clf_prob(y_test, probability):
  pred=np.where(probability > 0.63,1,0)
  confusion=confusion_matrix(y_test, pred)
  accuracy=accuracy_score(y_test,pred)
  precision=precision_score(y_test,pred) 
  recall=recall_score(y_test,pred) 
  # F1 스코어 추가 
  f1=f1_score(y_test,pred,average='macro')
  Roc_score=roc_auc_score(y_test,probability)
  print('임계값: ', 0.5) 
  print('오차행렬')
  print(confusion) 
  # f1 score print 추가 
  print('정확도: {0:.4f}, 정밀도: {1:.4f}, 재현율: {2:.4f}, F1:{3:.4f}, AUC:{4: .4f}'.format(accuracy,precision,recall,f1,Roc_score))

In [13]:
get_clf_prob(new_test_data_y,prob)

임계값:  0.5
오차행렬
[[510  20]
 [ 74 316]]
정확도: 0.8978, 정밀도: 0.9405, 재현율: 0.8103, F1:0.8931, AUC: 0.9537
